***GENERATED CODE FOR kmeans PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'sex'}], 'feature': 'sex', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '223', 'mean': '', 'stddev': '', 'min': 'female', 'max': 'male', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'sex'}, {'transformation_label': 'String Indexer', 'feature_label': 'sex'})
        transformationDF = transformationDF.drop('sex')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'age'}], 'feature': 'age', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '223', 'mean': '', 'stddev': '', 'min': '0s', 'max': '80s', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'age'}, {'transformation_label': 'String Indexer', 'feature_label': 'age'})
        transformationDF = transformationDF.drop('age')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'country'}], 'feature': 'country', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '223', 'mean': '', 'stddev': '', 'min': 'China', 'max': 'Korea', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'country'}, {'transformation_label': 'String Indexer', 'feature_label': 'country'})
        transformationDF = transformationDF.drop('country')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'province'}], 'feature': 'province', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '223', 'mean': '', 'stddev': '', 'min': 'Busan', 'max': 'Ulsan', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'province'}, {'transformation_label': 'String Indexer', 'feature_label': 'province'})
        transformationDF = transformationDF.drop('province')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'city'}], 'feature': 'city', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '223', 'mean': '', 'stddev': '', 'min': 'Andong-si', 'max': 'sankyeock-dong', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'city'}, {'transformation_label': 'String Indexer', 'feature_label': 'city'})
        transformationDF = transformationDF.drop('city')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'infection_case'}], 'feature': 'infection_case', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '223', 'mean': '', 'stddev': '', 'min': 'Bonghwa Pureun Nursing Home', 'max': 'overseas inflow', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'infection_case'}, {'transformation_label': 'String Indexer', 'feature_label': 'infection_case'})
        transformationDF = transformationDF.drop('infection_case')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'state'}], 'feature': 'state', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '223', 'mean': '', 'stddev': '', 'min': 'deceased', 'max': 'released', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'state'}, {'transformation_label': 'String Indexer', 'feature_label': 'state'})
        transformationDF = transformationDF.drop('state')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
import json
from sklearn.cluster import KMeans
import numpy as np
from sklearn import metrics
import pandas as pd


def kmeans(df, TrainingPercent, k):
    rows = df.count()
    percentrow = TrainingPercent / 100 * rows
    percentrows = int(percentrow)
    df = df.limit(percentrows)
    pd_df = df.toPandas()
    model = KMeans(n_clusters=k).fit(pd_df)
    labels = model.predict(pd_df)
    silhouette = metrics.silhouette_score(pd_df, labels, metric='euclidean')
    pd_df['prediction'] = pd.DataFrame(labels)
    labels_unique = np.unique(labels)
    totalClusters = len(labels_unique)
    display("totalClusters     : %s" % totalClusters)
    display(pd_df.head())
    return pd_df


class Clustering:

    def run(spark_DF, spark, config):
        stageAttributes = json.loads(config)
        stageAttributes['model']
        trainingPercent = stageAttributes['TrainingPercent']
        clusters = stageAttributes['totalClusters']

        bkmeans_model = kmeans(
            df=spark_DF, TrainingPercent=trainingPercent, k=clusters)


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run kmeansHooks.ipynb
try:
	#sourcePreExecutionHook()

	coronapatienttrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/TrainData/CoronaPatientTrain.csv', 'filename': 'CoronaPatientTrain.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(coronapatienttrain)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run kmeansHooks.ipynb
try:
	#transformationPreExecutionHook()

	manualfe = TransformationMain.run(coronapatienttrain,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "sex"}], "feature": "sex", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "223", "mean": "", "stddev": "", "min": "female", "max": "male", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "sex"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "age"}], "feature": "age", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "223", "mean": "", "stddev": "", "min": "0s", "max": "80s", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "age"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "country"}], "feature": "country", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "223", "mean": "", "stddev": "", "min": "China", "max": "Korea", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "country"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "province"}], "feature": "province", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "223", "mean": "", "stddev": "", "min": "Busan", "max": "Ulsan", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "province"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "city"}], "feature": "city", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "223", "mean": "", "stddev": "", "min": "Andong-si", "max": "sankyeock-dong", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "city"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "infection_case"}], "feature": "infection_case", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "223", "mean": "", "stddev": "", "min": "Bonghwa Pureun Nursing Home", "max": "overseas inflow", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "infection_case"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "state"}], "feature": "state", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "223", "mean": "", "stddev": "", "min": "deceased", "max": "released", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "state"}]}))

	#transformationPostExecutionHook(manualfe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run kmeansHooks.ipynb
try:
	#mlPreExecutionHook()

	kmean = Clustering.run(manualfe,spark,json.dumps( {"autoClustering": 0, "originalfile": "/FileStore/platform/sampleData/60dd9e965fc4a40b8094a2b0/part-00000-26edd22f-3312-4028-baf4-228a377f523d-c000.csv", "model": "K-Means", "TrainingPercent": 100, "totalClusters": 2}))

	#mlPostExecutionHook(kmean)

except Exception as ex: 
	logging.error(ex)
